In [35]:
import matplotlib.pyplot as plt
import tensorflow as tf
from imutils import paths
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout,Flatten,Dense,Input
from keras import Sequential
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from scipy.constants import lb
import numpy as np
import cv2 as cv
import math
import imutils
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import plot_confusion_matrix

In [3]:
def midpoint(ptA, ptB):
	return (ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5

In [4]:
def matchTemplatePrivate(img):
    METHOD = cv.TM_CCOEFF

    # lê novamente a imagem para evitar dados quebrados
    edged_img = cv.adaptiveThreshold(img, 255,
                                     cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    img2 = img.copy()

    # carrega template para joelho esquerdo e direito
    template_l = cv.imread("../templates/template_L.png", 0)
    template_r = cv.imread("../templates/template_R.png", 0)

    # encontra contornos
    edged_template_l = cv.adaptiveThreshold(template_r, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    edged_template_r = cv.adaptiveThreshold(template_l, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    w_l, h_l = template_l.shape[::-1]
    w_r, h_r = template_l.shape[::-1]

    # aplica o math template em ambas as imagens de template
    res_l = cv.matchTemplate(edged_img, edged_template_l, METHOD)
    res_r = cv.matchTemplate(edged_img, edged_template_r, METHOD)

    min_val_l, max_val_l, min_loc_l, max_loc_l = cv.minMaxLoc(res_l)
    min_val_r, max_val_r, min_loc_r, max_loc_r = cv.minMaxLoc(res_r)

    # define qual imagem deu melhor match
    if max_val_r > max_val_l:
        top_left = max_loc_r
        bottom_right = (top_left[0] + w_r, top_left[1] + h_r)
    else:
        top_left = max_loc_l
        bottom_right = (top_left[0] + w_l, top_left[1] + h_l)

    return top_left, bottom_right

In [5]:
def processImage(image):
    image = img_to_array(image, dtype='uint8')
    x, y = matchTemplatePrivate(image)
    image = image[x[1]:y[1],x[0]:y[0]]
    image = array_to_img(image)
    image = image.resize((224,224))
    image = img_to_array(image)
    #return cv.cvtColor(image,cv.COLOR_GRAY2RGB)
    return image

In [6]:
def find_distance_between_bones(img):
    gray = cv.GaussianBlur(img, (7, 7), 0)
    # perform edge detection, then perform a dilation + erosion to
    # close gaps in between object edges
    edged = cv.Canny(gray, 50, 100)
    edged = cv.dilate(edged, None, iterations=1)
    edged = cv.erode(edged, None, iterations=1)
    # find contours in the edge map
    cnts = cv.findContours(edged.copy(), cv.RETR_EXTERNAL,
        cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
    colors = ((0, 0, 255), (240, 0, 159), (0, 165, 255), (255, 255, 0),
        (255, 0, 255))
    refObj = None
    
    menorDistancia = math.inf
    distancias = 0
    cont = 0
    
    for c in cnts:
        # if the contour is not sufficiently large, ignore it
        #if cv.contourArea(c) < 10:
        #   continue
        # compute the rotated bounding box of the contour
        box = cv.minAreaRect(c)
        box = cv.cv.BoxPoints(box) if imutils.is_cv2() else cv.boxPoints(box)
        box = np.array(box, dtype="int")
        # order the points in the contour such that they appear
        # in top-left, top-right, bottom-right, and bottom-left
        # order, then draw the outline of the rotated bounding
        # box
        box = perspective.order_points(box)
        # compute the center of the bounding box
        cX = np.average(box[:, 0])
        cY = np.average(box[:, 1])
        # if this is the first contour we are examining (i.e.,
        # the left-most contour), we presume this is the
        # reference object
        if refObj is None:
            # unpack the ordered bounding box, then compute the
            # midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-right and
            # bottom-right
            (tl, tr, br, bl) = box
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)
            # compute the Euclidean distance between the midpoints,
            # then construct the reference object
            D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
            refObj = (box, (cX, cY), D / 224)
            continue
        # draw the contours on the image
        orig = img.copy()
        cv.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
        cv.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 2)
        # stack the reference coordinates and the object coordinates
        # to include the object center
        refCoords = np.vstack([refObj[0], refObj[1]])
        objCoords = np.vstack([box, (cX, cY)])
        # loop over the original points
        for ((xA, yA), (xB, yB), color) in zip(refCoords, objCoords, colors):
            
            # draw circles corresponding to the current points and
            # connect them with a line
            cv.circle(orig, (int(xA), int(yA)), 5, color, -1)
            cv.circle(orig, (int(xB), int(yB)), 5, color, -1)
            cv.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),
                color, 2)
            # compute the Euclidean distance between the coordinates,
            # and then convert the distance in pixels to distance in
            # units
            D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]

            distancias += D
            cont+=1
            if D< menorDistancia:
                menorDistancia = D

            (mX, mY) = midpoint((xA, yA), (xB, yB))
            cv.putText(orig, "{:.1f}in".format(D), (int(mX), int(mY - 10)),
                cv.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
            # show the output image
            #cv.imshow("Image", orig)
            #cv.waitKey(0)
    
    if cont == 0:
        mediaDistancia = 0
    else:
        mediaDistancia = (distancias/cont)
        
    if menorDistancia == math.inf:
        menorDistancia = 0
    return mediaDistancia, menorDistancia

In [7]:
tf.config.list_physical_devices('GPU')

[]

In [8]:
train_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\train_preprocessed"
val_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\val_preprocessed"
test_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\test_preprocessed"

In [9]:
train_images=list(paths.list_images(train_dataset))
val_images=list(paths.list_images(val_dataset))
test_images=list(paths.list_images(test_dataset))

In [10]:
train_data=[]
train_labels=[]
val_data=[]
val_labels=[]
test_data=[]
test_labels=[]

for i in train_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    train_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    train_data.append(image)

for i in val_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    val_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    val_data.append(image)

for i in test_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    test_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    test_data.append(image)

In [11]:
print(test_data[0].shape)
print(len(val_data))
print(len(val_labels))

(224, 224, 1)
1652
1652


In [12]:
train_data=np.array(train_data, dtype='uint8')
train_labels=np.array(train_labels)

val_data=np.array(val_data, dtype='uint8')
val_labels=np.array(val_labels)

test_data=np.array(test_data, dtype='uint8')
test_labels=np.array(test_labels)

In [13]:
train_data[0].shape

(224, 224, 1)

In [14]:
print(find_distance_between_bones(train_data[6346]))

(0, 0)


In [15]:
train_data_table = []
for data in train_data:
    media, menor = find_distance_between_bones(data)
    train_data_table.append((media,menor))

In [16]:
test_data_table = []
for data in test_data:
    media, menor = find_distance_between_bones(data)
    test_data_table.append((media,menor))

In [40]:
len(train_data_table)

11556

In [ ]:
cv.imshow("aaa",train_data[6346])
cv.imshow("bbb",train_data[6347])

cv.waitKey(0)
cv.destroyAllWindows()

In [41]:
train_labels

array(['0', '0', '0', ..., '4', '4', '4'], dtype='<U1')

In [22]:
type(train_data_table)

list

In [13]:
train_labels_cat = to_categorical(train_labels)

val_labels_cat = to_categorical(val_labels)

test_labels_cat = to_categorical(test_labels)

In [43]:
len(train_data_table)

11556

In [44]:
len(train_labels)

11556

In [15]:
train_labels_cat

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [17]:
clf = SVC(kernel='rbf').fit(train_data_table, train_labels)

In [18]:
clf_predict = clf.predict(test_data_table)

In [19]:
clf_accuracy = accuracy_score(test_labels, clf_predict)

In [20]:
print('Accuracy (RBF Kernel): ', "%.2f" % (clf_accuracy*100))

Accuracy (RBF Kernel):  38.59


In [26]:
train_data_np = np.array(train_data_table, dtype=object)

In [27]:
train_labels = np.array(train_labels, dtype=object)

In [28]:
# instanciar o modelo XGBoost
model = XGBClassifiers()
# chamar o fit para o modelo
history = model.fit(train_data_np, train_labels, verbose=False)

In [30]:
# fazer previsões em cima do dataset de teste
predictions = model.predict(test_data_table)


In [36]:
plot_confusion_matrix(model, train_data_table, test_data_table, cmap='Blues')

C:\Users\Windows 10\Documents\PUC\2022_2\PAI\PAI_ArtroseDetection\pai\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


ValueError: plot_confusion_matrix only supports classifiers